# Installation

In [12]:
!pip install lightgbm
!pip install eli5
!pip install shap
!pip install sklearn
!pip install pandas
!pip install numpy
!pip install tqdm

# Imports

In [3]:
import eli5
import lightgbm as lgb
import numpy as np
import os
import pandas as pd
import shap
import sklearn

from sklearn.metrics import auc, make_scorer
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from tqdm import tqdm

# Data

In [9]:
data_dir = 'data/'
csvs = [x for x in os.listdir(data_dir) if x.endswith('.csv')]
print(csvs)
for csv in csvs:
    print(csv)
    df = pd.read_csv(os.path.join(data_dir, csv))
    print(len(df))
    print(list(df))

['ADMISSIONS.csv', 'CALLOUT.csv', 'CAREGIVERS.csv', 'CPTEVENTS.csv', 'DATETIMEEVENTS.csv', 'DIAGNOSES_ICD.csv', 'DRGCODES.csv', 'D_CPT.csv', 'D_ICD_DIAGNOSES.csv', 'D_ICD_PROCEDURES.csv', 'D_ITEMS.csv', 'D_LABITEMS.csv', 'ICUSTAYS.csv', 'INPUTEVENTS_CV.csv', 'INPUTEVENTS_MV.csv', 'LABEVENTS.csv', 'MICROBIOLOGYEVENTS.csv', 'OUTPUTEVENTS.csv', 'PATIENTS.csv', 'PRESCRIPTIONS.csv', 'PROCEDUREEVENTS_MV.csv', 'PROCEDURES_ICD.csv', 'SERVICES.csv', 'TRANSFERS.csv']
ADMISSIONS.csv
58976
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS', 'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA']
CALLOUT.csv
34499
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SUBMIT_WARDID', 'SUBMIT_CAREUNIT', 'CURR_WARDID', 'CURR_CAREUNIT', 'CALLOUT_WARDID', 'CALLOUT_SERVICE', 'REQUEST_TELE', 'REQUEST_RESP', 'REQUEST_CDIFF', 'REQUEST_MRSA', '

c:\programdata\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (4,5,7,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


573146
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'COSTCENTER', 'CHARTDATE', 'CPT_CD', 'CPT_NUMBER', 'CPT_SUFFIX', 'TICKET_ID_SEQ', 'SECTIONHEADER', 'SUBSECTIONHEADER', 'DESCRIPTION']
DATETIMEEVENTS.csv


c:\programdata\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


4485937
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'CHARTTIME', 'STORETIME', 'CGID', 'VALUE', 'VALUEUOM', 'WARNING', 'ERROR', 'RESULTSTATUS', 'STOPPED']
DIAGNOSES_ICD.csv
651047
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE']
DRGCODES.csv
125557
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'DRG_TYPE', 'DRG_CODE', 'DESCRIPTION', 'DRG_SEVERITY', 'DRG_MORTALITY']
D_CPT.csv
134
['ROW_ID', 'CATEGORY', 'SECTIONRANGE', 'SECTIONHEADER', 'SUBSECTIONRANGE', 'SUBSECTIONHEADER', 'CODESUFFIX', 'MINCODEINSUBSECTION', 'MAXCODEINSUBSECTION']
D_ICD_DIAGNOSES.csv
14567
['ROW_ID', 'ICD9_CODE', 'SHORT_TITLE', 'LONG_TITLE']
D_ICD_PROCEDURES.csv
3882
['ROW_ID', 'ICD9_CODE', 'SHORT_TITLE', 'LONG_TITLE']
D_ITEMS.csv
12487
['ROW_ID', 'ITEMID', 'LABEL', 'ABBREVIATION', 'DBSOURCE', 'LINKSTO', 'CATEGORY', 'UNITNAME', 'PARAM_TYPE', 'CONCEPTID']
D_LABITEMS.csv
753
['ROW_ID', 'ITEMID', 'LABEL', 'FLUID', 'CATEGORY', 'LOINC_CODE']
ICUSTAYS.csv
61532
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID',

c:\programdata\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7,9,17,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


17527935
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'ITEMID', 'AMOUNT', 'AMOUNTUOM', 'RATE', 'RATEUOM', 'STORETIME', 'CGID', 'ORDERID', 'LINKORDERID', 'STOPPED', 'NEWBOTTLE', 'ORIGINALAMOUNT', 'ORIGINALAMOUNTUOM', 'ORIGINALROUTE', 'ORIGINALRATE', 'ORIGINALRATEUOM', 'ORIGINALSITE']
INPUTEVENTS_MV.csv
3618991
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTTIME', 'ENDTIME', 'ITEMID', 'AMOUNT', 'AMOUNTUOM', 'RATE', 'RATEUOM', 'STORETIME', 'CGID', 'ORDERID', 'LINKORDERID', 'ORDERCATEGORYNAME', 'SECONDARYORDERCATEGORYNAME', 'ORDERCOMPONENTTYPEDESCRIPTION', 'ORDERCATEGORYDESCRIPTION', 'PATIENTWEIGHT', 'TOTALAMOUNT', 'TOTALAMOUNTUOM', 'ISOPENBAG', 'CONTINUEINNEXTDEPT', 'CANCELREASON', 'STATUSDESCRIPTION', 'COMMENTS_EDITEDBY', 'COMMENTS_CANCELEDBY', 'COMMENTS_DATE', 'ORIGINALAMOUNT', 'ORIGINALRATE']
LABEVENTS.csv
27854055
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ITEMID', 'CHARTTIME', 'VALUE', 'VALUENUM', 'VALUEUOM', 'FLAG']
MICROBIOLOGYEVENTS.csv
631726
['ROW_ID', 'SUBJ

c:\programdata\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


4156450
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTDATE', 'ENDDATE', 'DRUG_TYPE', 'DRUG', 'DRUG_NAME_POE', 'DRUG_NAME_GENERIC', 'FORMULARY_DRUG_CD', 'GSN', 'NDC', 'PROD_STRENGTH', 'DOSE_VAL_RX', 'DOSE_UNIT_RX', 'FORM_VAL_DISP', 'FORM_UNIT_DISP', 'ROUTE']
PROCEDUREEVENTS_MV.csv
258066
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTTIME', 'ENDTIME', 'ITEMID', 'VALUE', 'VALUEUOM', 'LOCATION', 'LOCATIONCATEGORY', 'STORETIME', 'CGID', 'ORDERID', 'LINKORDERID', 'ORDERCATEGORYNAME', 'SECONDARYORDERCATEGORYNAME', 'ORDERCATEGORYDESCRIPTION', 'ISOPENBAG', 'CONTINUEINNEXTDEPT', 'CANCELREASON', 'STATUSDESCRIPTION', 'COMMENTS_EDITEDBY', 'COMMENTS_CANCELEDBY', 'COMMENTS_DATE']
PROCEDURES_ICD.csv
240095
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE']
SERVICES.csv
73343
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'TRANSFERTIME', 'PREV_SERVICE', 'CURR_SERVICE']
TRANSFERS.csv
261897
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'DBSOURCE', 'EVENTTYPE', 'PREV_CAREUNIT', 'CUR

In [17]:
df = pd.read_csv(os.path.join(data_dir, 'DRGCODES.csv'))
print(np.count_nonzero(~np.isnan(df['DRG_MORTALITY'].values)) / len(df['DRG_MORTALITY'].values))
print(len(df['SUBJECT_ID'].values))
print(len(np.unique(df['SUBJECT_ID'].values)))

0.5307071688555796
125557
46511


# Modeling

In [5]:
class BaseModel(object):
    """Abstract base model class."""

    def __init__(self, k_folds, tuning_metric, num_trials, confidence_level):
        """Initialize parameters."""
        self.k_folds = k_folds
        self.tuning_metric = tuning_metric
        self.confidence_level = confidence_level
        self.num_trials = num_trials

        self.eval_fn = sklearn.metrics.auc
        self.model = None
        self.model_fn = None
        self.best_params = None
        self.fixed_parameters = {}
        self.tuning_parameters = {}

    def train(self, x, y, feature_names=[]):
        """Train a model on the inputs and labels provided.

        Args:
            x (numpy.ndarray): Input data to train on
            y (numpy.ndarray): Labels corresponding to the input data
        """

        print(f'Input shape is {x.shape}')

        initial_parameters = self.fixed_parameters
        initial_parameters.update(
            {param: val[0]
                for param, val in self.tuning_parameters[0].items()})

        # Handle single output model instantiation
        self.model = self.model_fn(**initial_parameters)

        scoring_fn = make_scorer(
            self.eval_fn,
            greater_is_better=True)

        best_score = float('-inf')
        best_model = self.model
        folds = list(StratifiedKFold(self.k_folds).split(x, y))

        for i in tqdm(range(len(self.tuning_parameters))):
            if logger is not None:
                logger.log(f"Tuning step {i+1}:", self.tuning_parameters[i])

            # Perform a grid search over the parameters specified
            # in self.tuning_parameters[i].
            # Set verbose to 2 for printing each step of the grid search.
            # Set n_jobs to -1 to use all processors.
            grid_searcher = GridSearchCV(estimator=self.model,
                 param_grid=self.tuning_parameters[i],
                 scoring=scoring_fn,
                 iid=False,
                 cv=folds,
                 refit=False,
                 n_jobs=-1)
            grid_searcher.fit(x, y)

            # Record the entire CSV regardless for future reference
            results = grid_searcher.cv_results_
            df = pd.DataFrame(results)

            if grid_searcher.best_score_ > best_score:

                best_params = grid_searcher.best_params_
                best_score = grid_searcher.best_score_

                fixed_parameters = self.fixed_parameters
                fixed_parameters.update({param: val
                                         for param, val in best_params.items()})
                best_model = self.model_fn(**fixed_parameters)


        self.model = best_model

        # K-fold cross validation.
        sample_ids = np.zeros(x.shape[0])
        ground_truth = np.zeros(x.shape[0])
        predictions = np.zeros(x.shape[0])
        shap_values = np.zeros(x.shape)
        shap_interaction_values = np.zeros(shape=(x.shape[0], x.shape[1], x.shape[1]))
        expected_values = []
        for train_ids, test_ids in folds:
            x_train = x[train_ids]
            y_train = y[train_ids]
            self.model.fit(x_train, y_train)

            explanation = eli5.explain_weights_lightgbm(
                self.model,
                feature_names=feature_names,
                top=25)
            logger.log(eli5.format_as_text(explanation))

            x_test = x[test_ids]
            y_test = y[test_ids]
            y_pred = self.model.predict(x_test)

            sample_ids[test_ids] = test_ids
            ground_truth[test_ids] = y_test
            predictions[test_ids] = y_pred

            explainer = shap.TreeExplainer(self.model)
            shap_values[test_ids] = explainer.shap_values(x_test)
            shap_interaction_values[test_ids] = explainer.shap_interaction_values(x_test)
            expected_values.append(explainer.expected_value)

        shap_dir = logger.outputs_dir / 'shap'
        shap_dir.mkdir()

        expected_value = np.mean(expected_values)
        logger.log_output(expected_values, 'shap/expected_values')
        logger.log_output(shap_values, 'shap/shap_values')
        logger.log_output(shap_interaction_values,
                          'shap/shap_interaction_values')
        logger.log_output(x, 'shap/x')
        logger.log_output(feature_names, 'shap/feature_names')

        record = np.hstack((sample_ids.reshape(-1, 1),
                            ground_truth.reshape(-1, 1),
                            predictions.reshape(-1, 1)))
        print(f'Record shape is {record.shape}')

        # Bootstrapping to obtain confidence intervals.
        scores = []
        num_successes = 0
        num_tries = 0
        indices = list(range(len(ground_truth)))
        while (num_successes < self.num_trials):
            # Limit the number of tries.
            num_tries += 1
            if num_tries > 2 * self.num_trials:
                raise ValueError(
                    "Too many unsuccessful tries to compute metric.")

            # Handle case where only one class is included by indices.
            try:
                new_indices = np.random.choice(indices, size=len(indices))
                score = EVAL_FNS['C-STATS'](
                    ground_truth[new_indices], predictions[new_indices],
                    throw_error=True)
                scores.append(score)
                num_successes += 1
            except:
                pass

        mean = np.mean(scores)
        scores.sort()
        # Computed using the basic bootstrap
        lower = 2 * mean - scores[
            int(((1 + self.confidence_level) / 2) * num_successes)]
        upper = 2 * mean - scores[
            int(((1 - self.confidence_level) / 2) * num_successes)]
        logger.log(f'Lower bound: {lower}')
        logger.log(f'Mean: {mean}')
        logger.log(f'Upper bound: {upper}')

        cs = np.mean(cross_val_score(
            self.model, x, y, cv=folds, scoring=scoring_fn))
        logger.log(f'CV {self.tuning_metric}: {cs}')

    def get_folds(self, x, y):
        # TODO(ndass): try to clean this up
        values, counts = np.unique(y, return_counts=True)
        below_k = [i for i in np.argsort(counts) if counts[i] < self.k_folds]
        shifts = {}
        # Merge minority classes until there are more examples than folds.
        while below_k:
            i = below_k[0]

            # Find the closest merge location on the left.
            pos = -1
            while (i + pos) >= 0 and counts[i + pos] == 9999999999999:
                pos -= 1
            left = (
                counts[i + pos] if (i + pos) >= 0 else float("inf"), i + pos)

            # Find the closest merge location on the right.
            pos = 1
            while (i + pos) < len(values) and counts[i + pos] == 9999999999999:
                pos += 1
            right = (
                counts[i + pos] if (i + pos) < len(values) else float("inf"),
                i + pos)

            # Merge the current minority class with the class on the left or
            # right that has the fewest occurences.
            shift = min(left, right)[1]
            counts[shift] += counts[i]
            counts[i] = 9999999999999
            if values[i] in shifts.values():
                for key in shifts:
                    if shifts[key] == values[i]:
                        shifts[key] = values[shift]
            shifts[values[i]] = values[shift]

            # Recompute the classes that occur less than fold times.
            below_k = [
                i for i in np.argsort(counts) if counts[i] < self.k_folds]

        # Update the minority classes of y.
        values = list(values)
        y_temp = np.copy(y)
        for i in range(len(y_temp)):
            if counts[values.index(y_temp[i])] == 9999999999999:
                y_temp[i] = shifts[y_temp[i]]
        _, counts = np.unique(y_temp, return_counts=True)
        assert not [i for i in np.argsort(counts) if counts[i] < self.k_folds]

        # Run StratifiedKFold on the modified version of y to get proper
        # indices.
        return list(StratifiedKFold(self.k_folds).split(x, y_temp))

    def refit_on_full(self, train_x, train_y):
        """Refit model on full data.

        This is to avoid using refit=True flag on GridSearchCV.
        Before testing, we refit the best model on train+dev.

        Arguments:
            train_x: Input data to predict labels for
            train_y: Input labels
        """
        self.model.fit(train_x, train_y)

    def predict(self, x):
        """Predict labels for the inputs.

        Args:
            x (list[list[int]]): Input data to predict labels for

        Returns:
            (list[int]): The predicted labels for the input data
        """
        raise NotImplementedError('Subclass of BaseModel must implement '
                                  'predict.')

# Data

# Modeling

In [6]:
class SklearnModel(BaseModel):
    """Abstract sklearn model class."""

    def predict(self, x):
        """See BaseModel.predict."""
        return self.model.predict(x)

In [7]:
class LGBMRegressor(SklearnModel):
    """LightGBM regressor."""

    def __init__(self, k_folds, tuning_metric, num_trials, confidence_level,
                 **kwargs):
        super(LGBMRegressor, self).__init__(
            k_folds=k_folds, tuning_metric=tuning_metric,
            num_trials=num_trials, confidence_level=confidence_level)

        self.model_fn = lgb.LGBMRegressor

        # Set the hyperparameters to sweep with grid search.
        self.tuning_parameters = [{
            'min_child_samples': [0, 10, 20],
            # 'class_weight':['balanced'],
            # 'max_bin': [4, 64, 128],
            'max_depth': [2, 4, 8],
            'num_leaves': [3, 7, 15],
            # 'min_split_gain': [0, 0.1],
            'min_child_weight': [0],
            # 'reg_alpha': [0.],
            # 'reg_lambda': [0., 0.01],
            'n_estimators': [5, 10, 100],
        }]

In [8]:
class LGBMClassifier(SklearnModel):
    """LightGBM classifier."""

    def __init__(self, k_folds, tuning_metric, num_trials, confidence_level,
                 **kwargs):
        super(LGBMClassifier, self).__init__(
            k_folds=k_folds, tuning_metric=tuning_metric,
            num_trials=num_trials, confidence_level=confidence_level)

        self.model_fn = lgb.LGBMClassifier

        # Set the hyperparameters to sweep with grid search.
        self.tuning_parameters = [{
            'class_weight':['balanced'],
            #'learning_rate': [0.01, 0.1],
            # 'num_leaves': list(7 * 2**exp for exp in range(2, 7)),
            #'min_data_in_leaf': [25, 50, 100, 150],
            # 'max_depth': [1, 2, 3, 4, 5, 10],
            # 'n_estimators': [3, 5, 10, 30]
        }]